# 在岸美元兑人民币汇率和沪深300指数

In [1]:
recent_days = 3

import os
import pandas as pd

# 获取当前工作目录
cwd = os.getcwd()

# 判断是否存在figure文件夹，如果不存在就创建该文件夹
if not os.path.exists('figure'):
    os.makedirs('figure')

import pandas as pd

#USDCNY_table = pd.read_csv("USDCNY.CSV", parse_dates=True, encoding='gb2312')
#USDCNY_table = pd.read_csv("data_csv/USDCNY.CSV", parse_dates=True)
#USDCNY_table = USDCNY_table.loc[:,['日期', '开盘价(元)', '最高价(元)', '最低价(元)', '收盘价(元)']].dropna()
USDCNY_table = pd.read_csv("data_csv/USDCNY_bbg.CSV", parse_dates=True, skiprows=1)
USDCNY_table = USDCNY_table.loc[:,['Dates','Open','High','Low','Close']].dropna()
USDCNY_table.columns = ['Time','Open','High','Low','Close']
USDCNY_table['Time'] = pd.to_datetime(USDCNY_table['Time'])

#SH_table = pd.read_csv("data_csv/CSI300.CSV", parse_dates=True)
#SH_table = SH_table.loc[:,['日期', '开盘价(元)', '最高价(元)', '最低价(元)', '收盘价(元)']].dropna()
SH_table = pd.read_csv("data_csv/CSI300_bbg.CSV", parse_dates=True, skiprows=1)
SH_table = SH_table.loc[:,['Dates','Open','Close','High','Low']].dropna()
SH_table.columns = ['Time','Open','Close','High','Low']
SH_table['Time'] = pd.to_datetime(SH_table['Time'])

merged_table = pd.merge(SH_table, USDCNY_table, on='Time', how='inner')
merged_table.sort_values(by='Time', inplace=True)
merged_table

,Time,Open_x,Close_x,High_x,Low_x,Open_y,High_y,Low_y,Close_y
0,2023-02-03 09:31:00,4158.41,4152.64,4158.41,4152.12,6.7418,6.7418,6.7389,6.7389
1,2023-02-03 09:32:00,4153.12,4154.58,4155.58,4153.12,6.7395,6.7402,6.7395,6.7402
2,2023-02-03 09:33:00,4154.63,4150.29,4154.74,4149.82,6.7414,6.7429,6.7414,6.7429
3,2023-02-03 09:34:00,4150.15,4155.59,4155.59,4149.91,6.7431,6.7431,6.7428,6.7428
4,2023-02-03 09:35:00,4155.93,4153.36,4156.01,4151.46,6.7427,6.7431,6.7427,6.7431
...,...,...,...,...,...,...,...,...,...
31333,2023-08-18 14:54:00,3789.88,3788.63,3789.88,3788.63,7.2842,7.2842,7.2840,7.2840
31334,2023-08-18 14:55:00,3788.44,3786.26,3788.44,3786.26,7.2840,7.2847,7.2840,7.2847
31335,2023-08-18 14:56:00,3786.08,3786.20,3786.48,3785.63,7.2847,7.2847,7.2847,7.2847
31336,2023-08-18 14:57:00,3786.43,3786.53,3786.53,3786.43,7.2849,7.2854,7.2849,7.2854


In [2]:
from scipy.stats import pearsonr

#merged_table['Stock Returns'] = np.log(merged_table['Close_x'] / merged_table['Close_x'].shift(1))
#merged_table['FX Returns'] = np.log(merged_table['Close_y'] / merged_table['Close_y'].shift(1))
merged_table = merged_table.dropna()

r,pvalue = pearsonr(merged_table['Close_x'], merged_table['Close_y'])
r

-0.8168720545674648

In [3]:
#import matplotlib.pyplot as plt

# plt.rcParams['font.sans-serif'] = ['SimHei']
# plt.rcParams['axes.unicode_minus'] = False

import datetime as dt

end_date = merged_table['Time'].iloc[-1]
start_date = end_date - dt.timedelta(days=recent_days)
merged_table = merged_table[(merged_table['Time'] >= start_date) & (merged_table['Time'] <= end_date)]

from pyecharts.commons.utils import JsCode
from pyecharts.charts import Line
from pyecharts import options as opts

line = Line() #创建Line实例

# 设置折线1
line.add_xaxis(merged_table['Time'].tolist())
line.add_yaxis('在岸人民币汇率', merged_table['Close_y'].tolist(), label_opts=opts.LabelOpts(is_show=False), color='#FFB6C1')

# 设置折线2，并将yaxis_index设置为1，表示使用右侧y轴
line.add_yaxis('沪深300', merged_table['Close_x'].tolist(), yaxis_index=1, label_opts=opts.LabelOpts(is_show=False), color='#6495ED')

# 设置y轴1最大值、最小值和间隔
line.set_global_opts(
    yaxis_opts=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(4)}"
            )
        ),
        splitline_opts=opts.SplitLineOpts(is_show=True),
        min_=merged_table['Close_y'].mean() - 3.3*merged_table['Close_y'].std(),  # 左侧y轴最小值
        max_=merged_table['Close_y'].mean() + 3.3*merged_table['Close_y'].std(),  # 左侧y轴最大值
        #interval=0.1  # 左侧y轴间隔
    ),
    datazoom_opts=[
        opts.DataZoomOpts(
            is_show=True, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
            ),
            opts.DataZoomOpts(
                is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                ),
                ],
    tooltip_opts=opts.TooltipOpts(
        trigger="axis",
        axis_pointer_type="cross",
        background_color="rgba(245, 245, 245, 0.8)",
        border_width=1,
        border_color="#ccc",
        textstyle_opts=opts.TextStyleOpts(color="#000"),
        ),
    axispointer_opts=opts.AxisPointerOpts(
        is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
        ),
)

# 设置右侧y轴，并添加is_inverse=True参数
line.extend_axis(
    yaxis=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(2)}"
            )
        ),
        name='沪深300',
        min_=merged_table['Close_x'].mean() - 3.3*merged_table['Close_x'].std(),
        max_=merged_table['Close_x'].mean() + 3.3*merged_table['Close_x'].std(),
        #interval=100,
        position='right',
        is_inverse=True
    )
)

# 隐藏数据系列标签，使图表更加简洁
line.set_series_opts(label_opts=opts.LabelOpts(is_show=False))

line.render_notebook()

In [4]:
line.render('figure/CSI300-USDCNY.html')

'c:\\Users\\siruo\\Documents\\每周工作\\汇率股指相关性\\figure\\CSI300-USDCNY.html'

In [5]:
r,pvalue = pearsonr(merged_table['Close_x'], merged_table['Close_y'])
r

-0.2898533828209101

# 离岸美元兑人民币汇率和美元指数

In [6]:
#USDX_table = pd.read_csv("data_csv/USDX.CSV", parse_dates=True)
#USDX_table = USDX_table.loc[:,['日期', '开盘价(元)', '最高价(元)', '最低价(元)', '收盘价(元)']].dropna()
USDX_table = pd.read_csv("data_csv/USDX_bbg.CSV", parse_dates=True, skiprows=1)
USDX_table = USDX_table.loc[:,['Dates','Open','High','Low','Close']].dropna()
USDX_table.columns = ['Time','Open','High','Low','Close']
USDX_table['Time'] = pd.to_datetime(USDX_table['Time'])
#USDX_table['Time'] = pd.to_datetime(USDX_table['Time']) + dt.timedelta(hours=12) # 夏令时

#USDCNH_table = pd.read_csv("data_csv/USDCNH.CSV", parse_dates=True)
#USDCNH_table = USDCNH_table.loc[:,['日期', '开盘价(元)', '最高价(元)', '最低价(元)', '收盘价(元)']].dropna()
USDCNH_table = pd.read_csv("data_csv/USDCNH_bbg.CSV", parse_dates=True, skiprows=1)
USDCNH_table = USDCNH_table.loc[:,['Dates','Open','High','Low','Close']].dropna()
USDCNH_table.columns = ['Time','Open','High','Low','Close']
USDCNH_table['Time'] = pd.to_datetime(USDCNH_table['Time'])
#USDCNH_table['Time'] = pd.to_datetime(USDCNH_table['Time']) + dt.timedelta(hours=12)

merged_table2 = pd.merge(USDX_table, USDCNH_table, on='Time', how='outer')
merged_table2.sort_values(by='Time', inplace=True)
merged_table2

,Time,Open_x,High_x,Low_x,Close_x,Open_y,High_y,Low_y,Close_y
184340,2023-01-27 06:00:00,NaN,NaN,NaN,NaN,6.7371,6.7386,6.7367,6.7374
184341,2023-01-27 06:01:00,NaN,NaN,NaN,NaN,6.7376,6.7376,6.7368,6.7368
184342,2023-01-27 06:03:00,NaN,NaN,NaN,NaN,6.7374,6.7377,6.7374,6.7377
184343,2023-01-27 06:04:00,NaN,NaN,NaN,NaN,6.7378,6.7378,6.7375,6.7375
184344,2023-01-27 06:05:00,NaN,NaN,NaN,NaN,6.7374,6.7380,6.7370,6.7371
...,...,...,...,...,...,...,...,...,...
184335,2023-08-18 15:15:00,103.409,103.409,103.395,103.395,7.3057,7.3058,7.3042,7.3044
184336,2023-08-18 15:16:00,103.395,103.401,103.395,103.400,7.3044,7.3046,7.3041,7.3043
184337,2023-08-18 15:17:00,103.400,103.401,103.386,103.387,7.3043,7.3043,7.3034,7.3039
184338,2023-08-18 15:18:00,103.387,103.390,103.375,103.376,7.3039,7.3041,7.3029,7.3034


In [7]:
r,pvalue = pearsonr(merged_table2.dropna()['Close_x'], merged_table2.dropna()['Close_y'])
r

-0.1464942018109985

In [8]:
end_date = merged_table2['Time'].iloc[-1]
start_date = end_date - dt.timedelta(days=recent_days)
merged_table2 = merged_table2[(merged_table2['Time'] >= start_date) & (merged_table2['Time'] <= end_date)]


line = Line() #创建Line实例

# 设置折线1
line.add_xaxis(merged_table2['Time'].tolist())
line.add_yaxis('离岸人民币汇率', merged_table2['Close_y'].tolist(), label_opts=opts.LabelOpts(is_show=False), color='#FFB6C1')

# 设置折线2，并将yaxis_index设置为1，表示使用右侧y轴
line.add_yaxis('美元指数', merged_table2['Close_x'].tolist(), yaxis_index=1, label_opts=opts.LabelOpts(is_show=False), color='#6495ED')

# 设置y轴1最大值、最小值和间隔
line.set_global_opts(
    yaxis_opts=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(4)}"
            )
        ),
        splitline_opts=opts.SplitLineOpts(is_show=True),
        min_=merged_table2.dropna()['Close_y'].mean() - 3.4*merged_table2.dropna()['Close_y'].std(),  # 左侧y轴最小值
        max_=merged_table2.dropna()['Close_y'].mean() + 3.4*merged_table2.dropna()['Close_y'].std(),  # 左侧y轴最大值
        #interval=0.01  # 左侧y轴间隔
    ),
    datazoom_opts=[
        opts.DataZoomOpts(
            is_show=True, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
            ),
            opts.DataZoomOpts(
                is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                ),
                ],
    tooltip_opts=opts.TooltipOpts(
        trigger="axis",
        axis_pointer_type="cross",
        background_color="rgba(245, 245, 245, 0.8)",
        border_width=1,
        border_color="#ccc",
        textstyle_opts=opts.TextStyleOpts(color="#000"),
        ),
    axispointer_opts=opts.AxisPointerOpts(
        is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
        ),
)

# 设置右侧y轴，并添加is_inverse=True参数
line.extend_axis(
    yaxis=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(4)}"
            )
        ),
        name='美元指数',
        min_=merged_table2.dropna()['Close_x'].mean() - 3.4*merged_table2.dropna()['Close_x'].std(),
        max_=merged_table2.dropna()['Close_x'].mean() + 3.4*merged_table2.dropna()['Close_x'].std(),
        #interval=0.1,
        position='right',
        #is_inverse=True,
    )
)

# 隐藏数据系列标签，使图表更加简洁
line.set_series_opts(label_opts=opts.LabelOpts(is_show=False))

line.render_notebook()

In [9]:
line.render('figure/USDX-USDCNH.html')

'c:\\Users\\siruo\\Documents\\每周工作\\汇率股指相关性\\figure\\USDX-USDCNH.html'

In [10]:
r,pvalue = pearsonr(merged_table2.dropna()['Close_x'], merged_table2.dropna()['Close_y'])
r

0.3262481259841029

# 离岸美元兑人民币汇率和标准普尔500指数

In [11]:
#SPX_table = pd.read_csv("data_csv/SPX.GI.CSV", parse_dates=True)
#SPX_table = SPX_table.loc[:,['日期', '开盘价(元)', '最高价(元)', '最低价(元)', '收盘价(元)']].dropna()
SPX_table = pd.read_csv("data_csv/SPX.GI_bbg.CSV", parse_dates=True, skiprows=1)
SPX_table = SPX_table.loc[:,['Dates','Open','High','Low','Close']].dropna()
SPX_table.columns = ['Time','Open','High','Low','Close']
SPX_table['Time'] = pd.to_datetime(SPX_table['Time'])
#SPX_table['Time'] = pd.to_datetime(SPX_table['Time']) + dt.timedelta(hours=12) # 夏令时

merged_table3 = pd.merge(SPX_table, USDCNH_table, on='Time', how='inner')
merged_table3

,Time,Open_x,High_x,Low_x,Close_x,Open_y,High_y,Low_y,Close_y
0,2023-02-03 22:30:00,4136.69,4136.69,4128.40,4134.65,6.7716,6.7728,6.7709,6.7724
1,2023-02-03 22:31:00,4134.68,4135.12,4130.70,4135.12,6.7724,6.7743,6.7723,6.7733
2,2023-02-03 22:32:00,4135.29,4139.35,4134.89,4139.30,6.7733,6.7734,6.7700,6.7701
3,2023-02-03 22:33:00,4138.76,4139.34,4135.60,4136.47,6.7701,6.7702,6.7683,6.7686
4,2023-02-03 22:34:00,4136.82,4138.72,4135.92,4137.53,6.7686,6.7705,6.7686,6.7702
...,...,...,...,...,...,...,...,...,...
54561,2023-08-18 04:15:00,4370.36,4370.36,4370.36,4370.36,7.3040,7.3040,7.3040,7.3040
54562,2023-08-18 04:16:00,4370.36,4370.36,4370.36,4370.36,7.3040,7.3043,7.3040,7.3042
54563,2023-08-18 04:17:00,4370.36,4370.36,4370.36,4370.36,7.3042,7.3042,7.3038,7.3039
54564,2023-08-18 04:18:00,4370.36,4370.36,4370.36,4370.36,7.3039,7.3041,7.3039,7.3041


In [12]:
merged_table3 = merged_table3.dropna()

r,pvalue = pearsonr(merged_table3['Close_x'], merged_table3['Close_y'])
r

0.8505707094526449

In [13]:
end_date = merged_table3['Time'].iloc[-1]
start_date = end_date - dt.timedelta(days=recent_days)
merged_table3 = merged_table3[(merged_table3['Time'] >= start_date) & (merged_table3['Time'] <= end_date)]



line = Line() #创建Line实例

# 设置折线1
line.add_xaxis(merged_table3['Time'].tolist())
line.add_yaxis('离岸人民币汇率', merged_table3['Close_y'].tolist(), label_opts=opts.LabelOpts(is_show=False), color='#FFB6C1')

# 设置折线2，并将yaxis_index设置为1，表示使用右侧y轴
line.add_yaxis('标普500', merged_table3['Close_x'].tolist(), yaxis_index=1, label_opts=opts.LabelOpts(is_show=False), color='#6495ED')

# 设置y轴1最大值、最小值和间隔
line.set_global_opts(
    yaxis_opts=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(4)}"
            )
        ),
        splitline_opts=opts.SplitLineOpts(is_show=True),
        min_=merged_table3['Close_y'].mean() - 3.3*merged_table3['Close_y'].std(),  # 左侧y轴最小值
        max_=merged_table3['Close_y'].mean() + 3.3*merged_table3['Close_y'].std(),  # 左侧y轴最大值
        #interval=0.005  # 左侧y轴间隔
    ),
    datazoom_opts=[
        opts.DataZoomOpts(
            is_show=True, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
            ),
            opts.DataZoomOpts(
                is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                ),
                ],
    tooltip_opts=opts.TooltipOpts(
        trigger="axis",
        axis_pointer_type="cross",
        background_color="rgba(245, 245, 245, 0.8)",
        border_width=1,
        border_color="#ccc",
        textstyle_opts=opts.TextStyleOpts(color="#000"),
        ),
    axispointer_opts=opts.AxisPointerOpts(
        is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
        ),
)

# 设置右侧y轴，并添加is_inverse=True参数
line.extend_axis(
    yaxis=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(
            formatter=JsCode(
                "function(value){return value.toFixed(2)}"
            )
        ),
        name='标普500',
        min_=merged_table3['Close_x'].mean() - 3.3*merged_table3['Close_x'].std(),
        max_=merged_table3['Close_x'].mean() + 3.3*merged_table3['Close_x'].std(),
        #interval=10,
        position='right',
        # is_inverse=True,
    )
)

# 隐藏数据系列标签，使图表更加简洁
line.set_series_opts(label_opts=opts.LabelOpts(is_show=False))

line.render_notebook()

In [14]:
line.render('figure/SPX.GI-USDCNH.html')

'c:\\Users\\siruo\\Documents\\每周工作\\汇率股指相关性\\figure\\SPX.GI-USDCNH.html'

In [15]:
r,pvalue = pearsonr(merged_table3['Close_x'], merged_table3['Close_y'])
r

0.31987786142665664